In [2]:
import pandas as pd
import numpy as np 
from pathlib import Path
import json

import matplotlib.pyplot as plt 
from matplotlib.ticker import FuncFormatter
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler


from src.concrete.ml.sklearn import SGDClassifier

import warnings
warnings.filterwarnings('ignore')

In [3]:
N_ITERATIONS = 15
RANDOM_STATE = 42
X, y = datasets.load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

scaler = MinMaxScaler(feature_range=[-1, 1])
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

rng = np.random.default_rng(RANDOM_STATE)
perm = rng.permutation(x_train.shape[0])

x_train = x_train[perm, ::]
y_train = y_train[perm]

In [4]:
parameters_range = (-1.0, 1.0)

model_concrete = SGDClassifier(
    random_state=RANDOM_STATE,
    max_iter=N_ITERATIONS,
    fit_encrypted=True,
    parameters_range=parameters_range,
)

# Train with simulation on the full dataset
model_concrete.fit(x_train, y_train, fhe="execute")

model_concrete.compile(x_train)

dumped_model_path = Path("demo-data/classifier_fhe.json")

# Any kind of file-like object can be used 
with dumped_model_path.open("w") as f:
    # Dump the model in a file
    model_concrete.dump(f)

with open("demo-data/classifier_fhe.json", 'r') as j:
     contents = json.loads(j.read())

model = contents['type_name']
serialized_value = np.array(contents['serialized_value']['_q_weights']['serialized_value'])


# Measure accuracy on the test set using simulation
y_pred_fhe, proof = model_concrete.predict(x_test, model, serialized_value, fhe="execute")

accuracy_fhe = accuracy_score(y_test, y_pred_fhe)

print(f'The predictions are: {y_pred_fhe}\n The proof is: {proof}\n')
print(f'The accuracy is: {accuracy_fhe}')


The predictions are: [0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0
 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1
 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 The proof is: 3a6a23a078435d1115fda1d6a16b84bb57ea37b273164a09e99b8bbd5e3e715a

The accuracy is: 0.9005847953216374
